In [1]:
%matplotlib inline

import datetime


In [2]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
SOS_token = 0
EOS_token = 1


class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

The files are all in Unicode, to simplify we will turn Unicode
characters to ASCII, make everything lowercase, and trim most
punctuation.




In [4]:
# Turn a Unicode string to plain ASCII, thanks to
# http://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters


def normalizeString(s):
    s = unicodeToAscii(s.strip())
#     s = unicodeToAscii(s.lower().strip())
#     s = re.sub(r"([.!?])", r" \1", s)
#     s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s

To read the data file we will split the file into lines, and then split
lines into pairs. The files are all English → Other Language, so if we
want to translate from Other Language → English I added the ``reverse``
flag to reverse the pairs.




In [5]:
def readLangs(lang1, lang2, reverse=False):
    print("Reading lines...")

    # Read the file and split into lines
#     lines = open('headline_text.txt', encoding='utf-8').\
#         read().strip().split('\n')
    lines = open('eng-fra.txt', encoding='utf-8').\
        read().strip().split('\n')
    
    # Split every line into pairs and normalize
    pairs = [[normalizeString(s) for s in l.split('\t')] for l in lines]

    # Reverse pairs, make Lang instances
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)
    print("Done reading")
    return input_lang, output_lang, pairs

In [6]:
MAX_LENGTH = 10

eng_prefixes = (
    "i am ", "i m ",
    "he is", "he s ",
    "she is", "she s",
    "you are", "you re ",
    "we are", "we re ",
    "they are", "they re "
)

# def filterPair(p):
#     return True

def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and \
        len(p[1].split(' ')) < MAX_LENGTH 

# def filterPair(p):
#     return len(p[0].split(' ')) < MAX_LENGTH and \
#         len(p[1].split(' ')) < MAX_LENGTH and \
#         p[1].startswith(eng_prefixes)


def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

In [7]:
def prepareData(lang1, lang2, reverse=False):
    input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
    print("Read %s sentence pairs" % len(pairs))
    pairs = filterPairs(pairs)
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs


input_lang, output_lang, pairs = prepareData('headline', 'text', True)
print(random.choice(pairs))

Reading lines...
Done reading
Read 160872 sentence pairs
Trimmed to 135981 sentence pairs
Counting words...
Counted words:
text 39038
headline 22174
['Je me soucie de vous toutes.', 'I care about all of you.']


In [8]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [9]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input.cpu()).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.gru(output.cpu(), hidden.cpu())
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [11]:
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]


def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)


def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

In [12]:
def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    # number of tokens of the single sentence
    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)


    loss = 0

    # loop through each token in a single input sentence
    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(input_tensor[ei], encoder_hidden)

    # decoder takes a single token as input 
    decoder_input = torch.tensor([[SOS_token]], device=device)

    # pass the final hidden state of encoder
    decoder_hidden = encoder_hidden


    # loop over each token of the target sentence
    # decoder_input: single token
    for di in range(target_length):
        # decoder_output is a softmax vector over vocabulary size
        decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
        topv, topi = decoder_output.topk(1) 
        
        # squeeze to get rid of dimensions. decoder_input is a scalar tensor
        # it is important not to use the SOS, EOS in the language model
        # because topi is the location of the most likely token
        decoder_input = topi.squeeze().detach()  # detach from history as input

        # compare the loss of the current predicted token (decoder_output)
        # and the ground truth token
        loss += criterion(decoder_output.cpu(), target_tensor[di].cpu())
        
        # if the most likely token is EOS, stop
        if decoder_input.item() == EOS_token:
            break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length


In [13]:
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [14]:
def trainIters(encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
#     encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate)
#     decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate)

    training_pairs = [tensorsFromPair(random.choice(pairs))
                      for i in range(n_iters)]
    criterion = nn.NLLLoss()
    best_encoder = []
    best_decoder = []
    lowest_loss = float("inf")
    for iter in range(1, n_iters + 1):
        # select 1 pair -> input, output tensors
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        # train based on 1 pair of input and output tensor
        loss = train(input_tensor, target_tensor, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion)
        if loss < lowest_loss: 
            lowest_loss = loss
            best_encoder = encoder.state_dict()
            best_decoder = decoder.state_dict()
        print_loss_total += loss
        plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    showPlot(plot_losses)
    return encoder, decoder, encoder_optimizer, decoder_optimizer, loss

In [15]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np


def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

In [16]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        # single sentence as a tensor
        input_tensor = tensorFromSentence(input_lang, sentence)
        
        # num tokens
        input_length = input_tensor.size()[0]
        
        encoder_hidden = encoder.initHidden()

        # loop over each token of the input sequence
        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei], encoder_hidden)

        # use SOS as the decoder input token
        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        # feed the final hidden layer of encoder to the decoder
        decoder_hidden = encoder_hidden

        decoded_words = []

        # max length to sample
        for di in range(max_length):
            # Note: decoder_input is SOS at first
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden)
            
            # pick the most likely word
            topv, topi = decoder_output.data.topk(1)
            
            # if it is EOS then, stop. But add <EOS> to it. 
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                # else, store the actual token as output. 
                decoded_words.append(output_lang.index2word[topi.item()])
            
            # squeeze the most likely token to a scalar for the next round
            decoder_input = topi.squeeze().detach()

        return decoded_words


In [17]:
def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

In [18]:
hidden_size = 256
encoder1 = EncoderRNN(input_lang.n_words, hidden_size).to(device)
decoder1 = DecoderRNN(hidden_size, output_lang.n_words)

In [19]:
encoder, decoder, encoder_optimizer, decoder_optimizer, loss = \
            trainIters(encoder1, decoder1, n_iters=30000, print_every=100)

0m 7s (- 37m 37s) (100 0%) 5.1629
0m 13s (- 34m 28s) (200 0%) 4.6088
0m 20s (- 33m 41s) (300 1%) 4.4145
0m 26s (- 32m 54s) (400 1%) 4.2882
0m 32s (- 32m 22s) (500 1%) 3.9937
0m 39s (- 31m 55s) (600 2%) 4.1445
0m 46s (- 32m 10s) (700 2%) 4.4649
0m 53s (- 32m 36s) (800 2%) 4.7612
1m 0s (- 32m 45s) (900 3%) 4.4024
1m 7s (- 32m 41s) (1000 3%) 4.4594
1m 14s (- 32m 44s) (1100 3%) 4.2858
1m 21s (- 32m 41s) (1200 4%) 4.4093
1m 28s (- 32m 25s) (1300 4%) 4.1689
1m 34s (- 32m 8s) (1400 4%) 3.8790
1m 41s (- 32m 9s) (1500 5%) 4.2603
1m 48s (- 32m 6s) (1600 5%) 4.0298
1m 55s (- 32m 4s) (1700 5%) 4.2182
2m 2s (- 31m 58s) (1800 6%) 4.1023
2m 10s (- 32m 2s) (1900 6%) 4.4079
2m 16s (- 31m 54s) (2000 6%) 4.2154
2m 23s (- 31m 51s) (2100 7%) 4.2274
2m 31s (- 31m 54s) (2200 7%) 4.3189
2m 38s (- 31m 51s) (2300 7%) 4.2680
2m 45s (- 31m 46s) (2400 8%) 4.3076
2m 52s (- 31m 42s) (2500 8%) 4.4373
3m 0s (- 31m 36s) (2600 8%) 4.3593
3m 7s (- 31m 32s) (2700 9%) 4.3987
3m 14s (- 31m 27s) (2800 9%) 4.1092
3m 21s (- 31

29m 15s (- 10m 49s) (21900 73%) 3.9835
29m 24s (- 10m 41s) (22000 73%) 3.7871
29m 32s (- 10m 33s) (22100 73%) 3.8502
29m 41s (- 10m 25s) (22200 74%) 3.9597
29m 50s (- 10m 18s) (22300 74%) 3.9851
29m 59s (- 10m 10s) (22400 74%) 3.9049
30m 8s (- 10m 2s) (22500 75%) 4.1600
30m 16s (- 9m 54s) (22600 75%) 3.7916
30m 25s (- 9m 47s) (22700 75%) 3.8734
30m 34s (- 9m 39s) (22800 76%) 4.0059
30m 43s (- 9m 31s) (22900 76%) 3.8753
30m 52s (- 9m 23s) (23000 76%) 3.8507
31m 1s (- 9m 15s) (23100 77%) 4.0822
31m 9s (- 9m 7s) (23200 77%) 3.7238
31m 18s (- 9m 0s) (23300 77%) 3.9777
31m 27s (- 8m 52s) (23400 78%) 4.0022
31m 35s (- 8m 44s) (23500 78%) 3.8459
31m 44s (- 8m 36s) (23600 78%) 4.0362
31m 53s (- 8m 28s) (23700 79%) 3.9296
32m 2s (- 8m 20s) (23800 79%) 3.8906
32m 10s (- 8m 12s) (23900 79%) 3.7968
32m 19s (- 8m 4s) (24000 80%) 3.9403
32m 28s (- 7m 57s) (24100 80%) 3.8302
32m 37s (- 7m 49s) (24200 80%) 3.9566
32m 45s (- 7m 41s) (24300 81%) 3.8691
32m 55s (- 7m 33s) (24400 81%) 3.9611
33m 4s (- 7m 

In [25]:
evaluateRandomly(encoder1, decoder1)

> Je fais ca pour vivre.
= I do this for a living.
< I do for for <EOS>

> Que suis-je suppose faire ?
= What am I supposed to do?
< What do you to <EOS>

> Quel sport preferes-tu ?
= Which sport do you like the most?
< What is is of <EOS>

> J'ai a nouveau faim.
= I'm hungry again.
< I have to <EOS>

> Je n'ai pas apprecie mon instructrice.
= I didn't like my teacher.
< I didn't have my my <EOS>

> Puis-je te parler une seconde ?
= Can I talk to you a sec?
< Can I want to me <EOS>

> La facture se montait a 100 dollars.
= The bill amounted to 100 dollars.
< The is is to to to <EOS>

> Je dois m'y rendre seule.
= I must go alone.
< I have to to to <EOS>

> Tu peux te rendre ou bon te semble.
= You can go wherever you want to go.
< You can you to you <EOS>

> Elle n'a pas aime son mari.
= She didn't like her husband.
< She didn't his to his his <EOS>



In [22]:
def save_checkpoint(encoder, decoder, encoder_optimizer, decoder_optimizer,  loss, name="eng_fra_model.pt"):
    path = "./save/" + name
    torch.save({
                'encoder_model_state_dict': encoder.state_dict(),
                'decoder_model_state_dict': decoder.state_dict(),
                'encoder_optimizer_state_dict': encoder_optimizer.state_dict(),
                'decoder_optimizer_state_dict': decoder_optimizer.state_dict(),
                'timestamp': str(datetime.datetime.now()),
                'loss': loss,
                }, path)
save_checkpoint(encoder1, decoder1, encoder_optimizer, decoder_optimizer, loss)